#**Art Painting 이미지를 사용한 정확도 올리기**

#**<문제>**
- 제공된 훈련데이터를 사용하여 모델을 훈련한 후 테스트 데이터에 대한 예측 카테고리 값을 csv파일로 제출하세요.
- 아래의 훈련데이터만을 사용하여 각 카테고리별로 분류가 잘 되도록 신경망을 구성하세요.
- 각 카테고리 별 정답 값은 아래와 같습니다.
> 'dog' : 0  
 'elephant' : 1  
 'giraffe' : 2  
 'guitar' : 3  
 'horse' : 4  
 'house' : 5  
 'person' : 6

#**<목적>**
주어진 데이터를 활용하여 정확도를 최대로 높이세요.

#**<훈련/테스트 데이터 세트 설명>**
훈련/테스트 데이터는 `Step 2. 데이터 다운로드`에서 확인할 수 있습니다.

- 테스트 입력 데이터 파일 : test.zip
- 훈련 데이터 파일: train.zip
- 예시 출력 데이터 파일: test_answer(sample).csv

###**데이터 설명**
- 데이터 셋은 art painting으로 구성되어 있으며 7개의 카테고리로 구분되어 있습니다.
- 7개의 카테고리로는 classes에서 정의한 dog, elephant, giraffe, guitar, horse, house, person를 포함합니다.
- 하나의 이미지의 사이즈는 [3(Channel) * 227(Width) * 227(Height)]이며 각 카테고리 별 예시 이미지는 아래에서 확인할 수 있습니다.
- 훈련 데이터는 art painting 1698장이며 테스트 데이터는 art painting 350장입니다.
- 아래의 이미지는 순서대로 dog, elephant, giraffe, guitar, horse, house, person에 대한 예시입니다.
```
classes = ['dog', 'elephant', 'giraffe','guitar','horse','house','person']
```

#**<최종 제출 파일>**#
- **submission.csv**  
    1.최종 제출 파일은 label 필드를 가지는 csv파일이며 테스트 데이터의 label을 값으로 가져야 합니다.  
    2.최종 제출 파일은 test_answer(sample).csv 파일과 같은 크기여야 하고 label값을 채워야합니다.  

#**<데이터 관련 안내>**#
- `train` 데이터 안에는 각 라벨별로 폴더가 나뉘어져 있으며 각 폴더 안의 파일을 사용하여 학습하여야 합니다.
- `test` 데이터는 라벨별로 폴더가 나뉘어져있지 않으며 임의의 순서대로 이미지가 섞여있습니다.
- 최종 제출물의 경우 데이터의 순서가 오름차순으로 정렬되어 있습니다. 이 순서대로 각 이미지에 대한 라벨 값을 작성하여 제출하세요.

#**<채점 방식>**#
테스트 데이터에서 추정한 label 값을 사용해 정확도를 구하고 이를 기준으로 채점합니다.

In [ ]:
import os
import math
import zipfile
import torch
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

# Dataset, DataLoader 정의

In [ ]:
zipfile.ZipFile('./drive/MyDrive/Colab Notebooks/미술 작품 분류하기/train.zip').extractall()

classes = os.listdir('./train')

for dir in ['./data/train/','./data/val/']:
  for class_label in classes:
    os.makedirs(dir+class_label)

In [ ]:
for class_label in classes:
    cnt = int(len(os.listdir('./train/'+class_label)) * 0.8)
    train = os.listdir('./train/'+class_label)[:cnt]
    val = os.listdir('./train/'+class_label)[cnt:]
    
    for train_data in train:
        os.replace('./train/' + class_label + '/' +train_data, './data/train/'+ class_label +'/' + train_data)
    
    for val_data in val:
        os.replace('./train/' + class_label + '/' +val_data, './data/val/'+ class_label +'/' + val_data)

In [ ]:
transform = transforms.Compose([
                                transforms.CenterCrop(227),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                ])

data_dir = './data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), transform) for x in ['train', 'val']}
dataloaders = {x: DataLoader(
                       image_datasets[x],
                       batch_size=16, 
                       shuffle=True,
                       num_workers= 2) for x in ['train', 'val']}

# Model

In [ ]:
model = models.efficientnet_b7(pretrained=True)
model.fc = torch.nn.Linear(in_features=227, out_features=7, bias=True)
torch.nn.init.xavier_uniform_(model.fc.weight)
stdv = 1. / math.sqrt(model.fc.weight.size(1))
model.fc.bias.data.uniform_(-stdv, stdv)

Downloading: "https://download.pytorch.org/models/efficientnet_b7_lukemelas-dcc49843.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b7_lukemelas-dcc49843.pth


  0%|          | 0.00/255M [00:00<?, ?B/s]

tensor([-0.0267, -0.0465,  0.0152,  0.0506,  0.0651, -0.0230,  0.0616])

In [ ]:
from tqdm.notebook import tqdm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"{device} is using!")

model.to(device)
LEARNING_RATE = 0.0001
NUM_EPOCH = 10

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

cuda:0 is using!


In [ ]:
import torchsummary
torchsummary.summary(model, (3,227,227))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 114, 114]           1,728
       BatchNorm2d-2         [-1, 64, 114, 114]             128
              SiLU-3         [-1, 64, 114, 114]               0
            Conv2d-4         [-1, 64, 114, 114]             576
       BatchNorm2d-5         [-1, 64, 114, 114]             128
              SiLU-6         [-1, 64, 114, 114]               0
 AdaptiveAvgPool2d-7             [-1, 64, 1, 1]               0
            Conv2d-8             [-1, 16, 1, 1]           1,040
              SiLU-9             [-1, 16, 1, 1]               0
           Conv2d-10             [-1, 64, 1, 1]           1,088
          Sigmoid-11             [-1, 64, 1, 1]               0
SqueezeExcitation-12         [-1, 64, 114, 114]               0
           Conv2d-13         [-1, 32, 114, 114]           2,048
      BatchNorm2d-14         [-1, 32, 1

In [ ]:
# 캐시 비우기
torch.cuda.empty_cache()
# 안되면 batch size 줄이기

# Train

In [ ]:
best_test_accuracy = 0.
best_test_loss = 9999.

for epoch in range(NUM_EPOCH):
  for phase in ["train", "val"]:
    running_loss = 0.
    running_acc = 0.
    if phase == "train":
      model.train()
    elif phase == "val":
      model.eval()

    for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
      images = images.to(device)
      labels = labels.to(device)

      optimizer.zero_grad()

      with torch.set_grad_enabled(phase == "train"):
        logits = model(images)
        _, preds = torch.max(logits, 1)  
        loss = loss_fn(logits, labels)

        if phase == "train":
          loss.backward()
          optimizer.step()

      running_loss += loss.item() * images.size(0)
      running_acc += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(dataloaders[phase].dataset)
    epoch_acc = running_acc / len(dataloaders[phase].dataset)

    print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
    if phase == "val" and best_test_accuracy < epoch_acc:
      best_test_accuracy = epoch_acc
    if phase == "val" and best_test_loss > epoch_loss:
      best_test_loss = epoch_loss
print("학습 종료!")
print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}")

  0%|          | 0/85 [00:00<?, ?it/s]

현재 epoch-0의 train-데이터 셋에서 평균 Loss : 3.257, 평균 Accuracy : 0.505


  0%|          | 0/22 [00:00<?, ?it/s]

현재 epoch-0의 val-데이터 셋에서 평균 Loss : 0.782, 평균 Accuracy : 0.839


  0%|          | 0/85 [00:00<?, ?it/s]

...

현재 epoch-8의 val-데이터 셋에서 평균 Loss : 0.159, 평균 Accuracy : 0.950


  0%|          | 0/85 [00:00<?, ?it/s]

현재 epoch-9의 train-데이터 셋에서 평균 Loss : 0.035, 평균 Accuracy : 0.990


  0%|          | 0/22 [00:00<?, ?it/s]

현재 epoch-9의 val-데이터 셋에서 평균 Loss : 0.194, 평균 Accuracy : 0.941
학습 종료!
최고 accuracy : 0.9589442610740662, 최고 낮은 loss : 0.1275359703735872


# Test

In [ ]:
zipfile.ZipFile('./drive/MyDrive/Colab Notebooks/미술 작품 분류하기/test.zip').extractall()

In [ ]:
test_datasets = datasets.ImageFolder(os.path.join('./test'), transform)
test_loader = DataLoader(
                       test_datasets,
                       batch_size=16, 
                       num_workers= 2)

In [ ]:
import csv

model.eval()
preds = []
with torch.no_grad():
    for step, (inputs, _) in enumerate(tqdm(test_loader)):
        inputs = inputs.to(device)
        outputs = model(inputs)
        pred = torch.softmax(outputs, dim=1).detach().squeeze().cpu().numpy()
        for p in pred:
            preds.append(np.argmax(p))

with open('./result.csv', 'w', encoding='utf-8-sig', newline='') as f:
    wr = csv.writer(f)
    wr.writerow(['','answer value'])
    for idx, pred in enumerate(preds):
        wr.writerow([idx, pred])

100%|██████████| 22/22 [00:04<00:00,  5.17it/s]
